In [2]:
# Test script for execute_kql_query function
from azure.identity import DefaultAzureCredential
from azure.monitor.query import LogsQueryClient, LogsQueryStatus
from utils.kql_query import execute_kql_query
from dotenv import load_dotenv
import os
import pandas as pd

# Initialize credentials and client
load_dotenv()
credential = DefaultAzureCredential()
client = LogsQueryClient(credential=credential)
workspace_id = os.getenv("SENTINEL_WORKSPACE_ID")

# Check if workspace_id is set, if set then good to go, else raise error
if not workspace_id:
    raise ValueError("SENTINEL_WORKSPACE_ID environment variable is not set.")


In [11]:
# Set investigation file path
investigation_file_path = os.path.join("investigations", "example-case")
# Set investigation config file path
investigation_config_path = os.path.join(investigation_file_path, "config.yaml")
# Set KQL query file path
kql_query_file_path = os.path.join(".", "queries", "mitre", "command-and-control", "command-and-control.kql")

# Print config and kql file paths
print(f"Investigation config file path: {investigation_config_path}")
print(f"KQL query file path: {kql_query_file_path}")

# Test to ensure config file and KQL query file exist
assert os.path.exists(investigation_config_path), f"Config file not found at {investigation_config_path}"
assert os.path.exists(kql_query_file_path), f"KQL query file not found at {kql_query_file_path}"

Investigation config file path: investigations\example-case\config.yaml
KQL query file path: .\queries\mitre\command-and-control\command-and-control.kql


In [ ]:
from utils.config_loader import load_config
from utils.query_template import render_kql_file

# Read in config file
config = load_config(str(investigation_config_path))

# Read in and render KQL file
rendered_query = render_kql_file(str(kql_query_file_path), config)

print(rendered_query)

// ============================================================================
// Command and Control (C2) Detection Queries
// ============================================================================
// Description: Hunt for C2 beaconing, tunneling, and communication patterns
// Data Sources: CommonSecurityLog, DeviceNetworkEvents, DnsEvents
// MITRE ATT&CK: T1071, T1090, T1095 (Command and Control)
// ============================================================================

// ----------------------------------------------------------------------------
// 1. Beaconing Detection - Regular Network Connections
// ----------------------------------------------------------------------------
// Detects regular, consistent network connections indicative of C2 beaconing
let timeWindow = 24h;
let beaconThreshold = 10;
CommonSecurityLog
| where TimeGenerated > ago(timeWindow)
| where DeviceAction != "Deny"
| extend ConnectionTime = bin(TimeGenerated, 1m)
| summarize 
    ConnectionCou

In [ ]:
# Define and execute query
kql_query = """
DeviceProcessEvents
| sample 10
"""

resp = client.query_workspace(workspace_id, kql_query, timespan=None)

if resp.status == LogsQueryStatus.PARTIAL:
    table = resp.partial_data[0]
elif resp.status == LogsQueryStatus.SUCCESS:
    table = resp.tables[0]
else:
    raise RuntimeError("Query failed")

df = pd.DataFrame(table.rows, columns=table.columns)
df.head()

,TenantId,AccountDomain,AccountName,AccountObjectId,AccountSid,AccountUpn,ActionType,AdditionalFields,AppGuardContainerId,DeviceId,...,ProcessRemoteSessionDeviceName,ProcessRemoteSessionIP,InitiatingProcessSessionId,IsInitiatingProcessRemoteSession,InitiatingProcessRemoteSessionDeviceName,InitiatingProcessRemoteSessionIP,InitiatingProcessUniqueId,ProcessUniqueId,SourceSystem,Type
0,19864e56-23ed-497a-b56f-d7aa24c8a6f2,nt authority,system,,S-1-5-18,,ProcessCreated,None,,711c7de5324016bb0face48b338c3fe467923c4d,...,,,0,False,,,12103423998558271,12103423998566239,,DeviceProcessEvents
1,19864e56-23ed-497a-b56f-d7aa24c8a6f2,nt authority,system,,S-1-5-18,,ProcessCreated,None,,c3b4e2b38bfc3e3bab76b7eb6c1f26b9a078648c,...,,,0,False,,,10977524091726953,10977524091726955,,DeviceProcessEvents
2,19864e56-23ed-497a-b56f-d7aa24c8a6f2,nt authority,system,,S-1-5-18,,ProcessCreated,None,,711c7de5324016bb0face48b338c3fe467923c4d,...,,,0,False,,,12103423998558219,12103423998566241,,DeviceProcessEvents
3,19864e56-23ed-497a-b56f-d7aa24c8a6f2,nt authority,system,,S-1-5-18,,ProcessCreated,None,,d8bcc72637cbfa16b8512c167e0961cf4d389ea8,...,,,0,False,,,,,,DeviceProcessEvents
4,19864e56-23ed-497a-b56f-d7aa24c8a6f2,nt authority,system,,S-1-5-18,,ProcessCreated,None,,8fd1c882018b1156f864c5a91f0b408ffa8f9784,...,,,0,False,,,,,,DeviceProcessEvents
